In [ ]:
from tvDatafeed import TvDatafeed, Interval
import pandas as pd
import os
import json
import time

# Configuration
username = "truthfulOwl34362"
password = "asdfasdf12#$jkl;"

mapping_file = "D:/Programming/Download_Backtest_Deploy_data/1__Download/TradingView_data_download/symbol_mappings.json"

nifty_50 = ['ITCHOTELS', 'INDIGO', 'MAXHEALTH', 'ADANIENT', 'ADANIPORTS', 'APOLLOHOSP', 'ASIANPAINT', 'AXISBANK', 'BAJAJFINSV', 'BAJAJ_AUTO', 'BAJFINANCE', 'BEL', 'BHARTIARTL', 'BPCL', 'BRITANNIA', 'CIPLA', 'COALINDIA', 'DIVISLAB', 'DRREDDY', 'EICHERMOT', 'ETERNAL', 'GAIL', 'GRASIM', 'HCLTECH', 'HDFCBANK', 'HDFCLIFE', 'HEROMOTOCO', 'HINDALCO', 'HINDUNILVR', 'ICICIBANK', 'INDUSINDBK', 'INFY', 'IOC', 'ITC', 'JIOFIN', 'JSWSTEEL', 'KOTAKBANK', 'LTIM', 'LT', 'M&M', 'MARUTI', 'NESTLEIND', 'NTPC', 'ONGC', 'POWERGRID', 'RELIANCE', 'SBILIFE', 'SBIN', 'SHREECEM', 'SHRIRAMFIN', 'SUNPHARMA', 'TATACONSUM', 'TATAMOTORS', 'TATASTEEL', 'TCS', 'TECHM', 'TITAN', 'TRENT', 'ULTRACEMCO', 'UPL', 'WIPRO', 'YESBANK']


In [ ]:
# Symbol mapping management
def load_symbol_mappings():
    try:
        with open(mapping_file, 'r') as f:
            return json.load(f)
    except FileNotFoundError:
        return {}

def save_symbol_mappings(mappings):
    with open(mapping_file, 'w') as f:
        json.dump(mappings, f, indent=2)

In [ ]:
def download_stock_data(symbol, resolution, exchange='NSE', n_bars=1188):
    try:
        tv = TvDatafeed(username=username, password=password)
        data = tv.get_hist(symbol=symbol, exchange=exchange, interval=resolution, n_bars=n_bars)
        
        if data is None or data.empty:
            return None
            
        data.reset_index(inplace=True)
        data = data[['datetime', 'open', 'high', 'low', 'close', 'volume']]
        data.columns = ['Datetime', 'Open', 'High', 'Low', 'Close', 'Volume']
        data['Datetime'] = data['Datetime']
        
        for col in ['Open', 'High', 'Low', 'Close']:
            data[col] = data[col].round(2)
            
        return data
    except:
        print("Error in downloading data for symbol:", symbol)
        return None

In [ ]:
def master_download(resolution):

    resolution_map = {
        "1": Interval.in_1_minute, "3": Interval.in_3_minute, "5": Interval.in_5_minute, "15": Interval.in_15_minute,       # Minutes based
        "30": Interval.in_30_minute, "60": Interval.in_1_hour, "120": Interval.in_2_hour, "240": Interval.in_4_hour,
        "D": Interval.in_daily                                                                                              # Daily         
        }
    
    if resolution not in resolution_map: raise ValueError(f"Invalid resolution: {resolution}. Choose from {list(resolution_map.keys())}")
    resolution_tv = resolution_map["D"]

    storage_folder = f"D:/Programming/Download_Backtest_Deploy_data/1__Download/TradingView_data_download/storage_TV_{resolution}"
    os.makedirs(storage_folder, exist_ok=True)

    mappings = load_symbol_mappings()
    successful = []
    failed = []
    
    for i, original_symbol in enumerate(nifty_50, 1):
        print(f"\n[{i}/{len(nifty_50)}] Processing: {original_symbol}")
        
        # Check if file already exists
        filename = f"{storage_folder}/{original_symbol}_TV_{resolution}.csv"
        if os.path.exists(filename):
            print(f"Already exists: {original_symbol}")
            successful.append(original_symbol)
            continue
            
        success = False
        
        # Try original symbol first
        data = download_stock_data(original_symbol, resolution_tv)
        
        if data is not None:
            data.to_csv(filename, index=False)
            successful.append(original_symbol)
            success = True
        else:
            # Try alternatives from JSON
            if original_symbol in mappings and isinstance(mappings[original_symbol], list):
                for alt_symbol in mappings[original_symbol]:
                    data = download_stock_data(alt_symbol, resolution_tv)
                    if data is not None:
                        data.to_csv(filename, index=False)
                        mappings[original_symbol] = alt_symbol
                        save_symbol_mappings(mappings)
                        successful.append(original_symbol)
                        success = True
                        break
            
            # User input for failed symbols
            if not success:
                while True:
                    user_input = input(f"\nSymbol '{original_symbol}' failed. Options:\n1. Enter correct symbol\n2. Skip this symbol\nChoice (1/2): ")
                    
                    if user_input == '2':
                        failed.append(original_symbol)
                        break
                    elif user_input == '1':
                        new_symbol = input(f"Enter correct symbol for {original_symbol}: ").strip().upper()
                        if new_symbol:
                            data = download_stock_data(new_symbol, resolution_tv)
                            if data is not None:
                                data.to_csv(filename, index=False)
                                mappings[original_symbol] = new_symbol
                                save_symbol_mappings(mappings)
                                successful.append(original_symbol)
                                print(f"Success with: {new_symbol}")
                                break
                            else:
                                print(f"Failed with: {new_symbol}")
        
        time.sleep(1)
    
    print(f"\n=== Summary ===")
    print(f"Successful: {len(successful)}/{len(nifty_50)}")
    print(f"Failed: {failed}")
    
    return successful, failed

# Run master download
successful, failed = master_download(resolution="D")